In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import scipy.io as sio
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection

In [ ]:
colors_networks = np.array([
    "#FF0000",  # Red DMN
    "#00FF00",  # Green VIS
    "#0000FF",  # Blue FP
    "#FFFF00",  # Yellow Reward
    "#FF00FF",  # Magenta DA
    "#000000",  # nan
    "#00FFFF",  # Cyan VA
    "#800080",  # Purple SN
    "#FFA500",  # Orange CO
    "#008000",  # Dark Green SM Dorsal
    "#800000",  # Maroon SM Lateral
    "#008080",  # Teal AUD
    "#000000",  # nan
    "#000000",  # nan
    "#000080",  # Navy ParietoMedial
    "#000000",  # nan
    "#FFD700",  # Gold Medial Temporal Lobe
    "#000000",  # nan
])

func_network_affiliation = np.loadtxt(f"{MAIN_DIR}/scripts/func_networks/masks/ROIs_300inVol_MNI/ROIs_CommunityAffiliation.txt").astype(int)
func_network_unique_list = np.unique(func_network_affiliation)

# colores para los grafos con ROIs
node_colors = []
for iRoi in range(300):
    if func_network_affiliation[iRoi] == 2 or func_network_affiliation[iRoi] == 3 or func_network_affiliation[iRoi] == 5 or func_network_affiliation[iRoi] == 7 or func_network_affiliation[iRoi] == 10 or func_network_affiliation[iRoi] == 11:       
        node_colors.append(colors_networks[func_network_affiliation[iRoi]-1])
    else:
        node_colors.append("#cccccc")

In [ ]:
def draw_graph(condition, minval, maxval, output_file):
    NODE_SIZE = 9000
    FONT_SIZE = 35
    MIN_EDGE_WIDTH = 10
    MAX_EDGE_WIDTH = 40

    values = []
    from_array = []
    to_array = []
    for i in range(len(network_list)):
        for j in range(i+1, len(network_list)):
            from_array.append(network_names[i])
            to_array.append(network_names[j])
            values.append(condition[i][j])

    #print(len(from_array), len(to_array), len(values))
    df = pd.DataFrame({"from": from_array, "to": to_array, "value": values})
    G = nx.from_pandas_edgelist(df, "from", "to", create_using=nx.Graph())

    values_positive = df["value"][df["value"] >= 0]
    values_negative = df["value"][df["value"] < 0]

    edge_list = list(map(lambda value: np.interp(value, [minval, maxval], [0, 1]), df["value"]))
    width_list = list(map(lambda value: np.interp(np.abs(value), [0, maxval], [MIN_EDGE_WIDTH, MAX_EDGE_WIDTH]), values))
    alpha_list = list(map(lambda value: np.interp(np.abs(value), [0, maxval], [0.1, 1.0]), values))
    
    
    colormap = plt.cm.get_cmap('RdBu_r')
    fig, ax = plt.subplots(1, figsize=(10,10))
    pos = nx.kamada_kawai_layout(G)
    nodes = nx.draw_networkx_nodes(G, pos, node_size=NODE_SIZE, node_color='black')
    nx.draw_networkx_labels(G, pos, font_color="white", font_size=FONT_SIZE, font_weight="bold")

    for i, edge in enumerate(G.edges()):
        nx.draw_networkx_edges(G, pos, edgelist=[edge], alpha=alpha_list[i], width=width_list[i], edge_color=colormap(edge_list[i]))

    ax.set_facecolor('#ffffff')
    ax.axis('off')
    fig.set_facecolor('#ffffff')
    
    plt.tight_layout()
    plt.savefig(output_file)

def draw_roi_graph(cr, size=(8,8), output_file=""):
    plt.subplots(1, figsize=size)

    c = cr.mean(axis=0)
    c[np.eye(300).astype(bool)] = 0
    G = nx.from_numpy_array(c > np.percentile(c, 99))
    pos = nx.kamada_kawai_layout(G)
    nx.draw(G, pos, edge_color="#888888", node_color=node_colors, node_size=150)

    plt.savefig(output_file)

In [ ]:
MAIN_DIR = "/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn"
network_names = ["DMN", "VIS", "FP", "Rew", "DA", "VA", "SN", "CO", "SMD", "SML", "AUD", "PM", "MTL", "un"]
network_list = list(range(13))
min_val = -0.15
max_val = 0.15

connectivity = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_2.mat")["C_net_2"]

transicion_diff_conn = np.mean(connectivity[:,1,:,:]-connectivity[:,0,:,:], axis=0)[network_list][:,network_list]
alteracion_diff_conn = np.mean(connectivity[:,2,:,:]-connectivity[:,0,:,:], axis=0)[network_list][:,network_list]
recuperacion_diff_conn = np.mean(connectivity[:,3,:,:]-connectivity[:,0,:,:], axis=0)[network_list][:,network_list]

In [ ]:
connectivity_regions = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_300.mat")["C"]

connectivity_per_network = connectivity.mean(axis=3)
connectivity_per_network_trans = connectivity_per_network[:,1,:] - connectivity_per_network[:,0,:]
connectivity_per_network_alter = connectivity_per_network[:,2,:] - connectivity_per_network[:,0,:]
connectivity_per_network_recup = connectivity_per_network[:,3,:] - connectivity_per_network[:,0,:]

def calculate_statistics_network(condicion):
    if condicion == "trans":
        connectivity_per_network = connectivity_per_network_trans
    if condicion == "alter":
        connectivity_per_network = connectivity_per_network_alter
    if condicion == "recup":
        connectivity_per_network = connectivity_per_network_recup

    data = {"sesion": [], "network": [], "diff": []}
    for iSess in range(20):
        for iNetwork in range(13):
            data["sesion"].append(iSess+1)
            data["network"].append(iNetwork+1)
            data["diff"].append(connectivity_per_network[iSess, iNetwork])

    df = pd.DataFrame(data)
    df['network'] = df['network'].astype('category')
    df['sesion'] = df['sesion'].astype('category')

    md = smf.mixedlm("diff ~ network - 1", df, groups=df["sesion"])
    mdf = md.fit()
    mdf.summary()

    #h, pcorregido = fdrcorrection(mdf.pvalues[:-1].values, alpha=0.05, method="indep", is_sorted=False)
    #return mdf.pvalues[:-1][h]

    return mdf.pvalues[:-1][mdf.pvalues[:-1] < 0.05]

In [ ]:
print(calculate_statistics_network("trans"))
print(calculate_statistics_network("alter"))
print(calculate_statistics_network("recup"))

#### Figura grafos

In [ ]:
draw_graph(transicion_diff_conn, min_val, max_val, output_file=f"{MAIN_DIR}/scripts/figuras-finales/conectividad-redes/figuras/grafo_transicion.png")

In [ ]:
draw_graph(alteracion_diff_conn, min_val, max_val, output_file=f"{MAIN_DIR}/scripts/figuras-finales/conectividad-redes/figuras/grafo_alteracion.png")

In [ ]:
draw_graph(recuperacion_diff_conn, min_val, max_val, output_file=f"{MAIN_DIR}/scripts/figuras-finales/conectividad-redes/figuras/grafo_recuperacion.png")

In [ ]:
plt.subplots(1, figsize=(8,8))
plt.imshow(np.random.rand(10,10), cmap="RdBu_r", vmin=-0.15, vmax=0.15)
plt.colorbar()

plt.savefig("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-redes/figuras/colorbar.png")

In [ ]:
draw_roi_graph(connectivity_regions[:,0,:,:], size=(8,8), output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-redes/figuras/grafo_roi_reposo.png")

In [ ]:
draw_roi_graph(connectivity_regions[:,1,:,:], size=(10,8), output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-redes/figuras/grafo_roi_transicion.png")

In [ ]:
draw_roi_graph(connectivity_regions[:,2,:,:], size=(14,8), output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-redes/figuras/grafo_roi_alteracion.png")

In [ ]:
draw_roi_graph(connectivity_regions[:,3,:,:], size=(10,8), output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-redes/figuras/grafo_roi_recuperacion.png")